# 0. Setup

Load the modules for training.

In [ ]:
%matplotlib inline

import os
os.environ["CUDA_VISIBLE_DEVICES"]= "3" 

import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow 
from tensorflow.keras import layers
from keras.models import Model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import pandas as pd
import random
from keras import backend as K

from keras.models import Sequential
import warnings
warnings.filterwarnings(action='ignore')

# 1. Laoding the dataset

- Load the preprocessed dataset
- The size of the dataset is (R, L). In our setting, we define R, L = 224, 224
- The first session contains 336 subjects with 6 fingerprint images
- The first session contains 160 subjects with 6 fingerprint images

In [ ]:
R, L = 224, 224

BATCH_SIZE = 32
INPUT_SIZE = (R, L, 1)

train_seq = iaa.Sequential([
    iaa.GaussianBlur(sigma=(0, 0.7)),
    iaa.Dropout((0.01, 0.15), per_channel=0.5),
    iaa.Affine(
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
        rotate=(-30, 30),
        order=[0, 1],
        cval=1
    )
], random_order=True)

test_seq = iaa.Sequential([
    iaa.GaussianBlur(sigma=(0, 0.7)),
    iaa.Dropout((0.01, 0.15), per_channel=0.5),
    iaa.Affine(
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
        rotate=(-30, 30),
        order=[0, 1],
        cval=1
    )
], random_order=True)
 

# 2. Loading the Dataset
- Load the data and split dataset with training, validation, and test.
- Change the type as float32

In [ ]:
SOKOTO_DATA_PATH = 'Define the stored data path'
DATA_SIZE = 6000
x_real = np.load(SOKOTO_DATA_PATH)  / 255.
y_real = np.zeros(DATA_SIZE, dtype=int)
for i in range(DATA_SIZE):
    y_real[i] = i
    
x_train, x_test, label_train, label_test = train_test_split(x_real, y_real, test_size=0.2, random_state = 42)
x_train, x_val, label_train, label_val = train_test_split(x_train, label_train, test_size=0.25, random_state = 42)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

# 3. Data Generator

In [ ]:
class DataGenerator(tensorflow.keras.utils.Sequence):
    def __init__(self, x, label, x_real, y_real, batch_size=4, shuffle=True):
        self.x = x
        self.label = label
        self.x_real = x_real
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        

    def __len__(self):
        return int(np.floor(len(self.x) / self.batch_size))

    def __getitem__(self, index):
        x1_batch = self.x[index*self.batch_size:(index+1)*self.batch_size]
        label_batch = self.label[index*self.batch_size:(index+1)*self.batch_size]
        
        x2_batch = np.empty((self.batch_size, R, L, 1), dtype=np.float32)
        y_batch = np.zeros((self.batch_size, 1), dtype=np.float32)
        
        x1_batch, x2_batch = np.array(x1_batch), np.array(x2_batch)
        
        if self.shuffle:
            x1_batch = train_seq.augment_images(x1_batch)
        
        for i, idx in enumerate(label_batch):
            if random.random() > 0.5:
                x2_batch[i] = self.x_real[idx]
                y_batch[i] = 1.
            else:
                not_chosen_idx = random.choice(list(set(y_real) - set([idx])))
                x2_batch[i] = self.x_real[not_chosen_idx]
                y_batch[i] = 0.

        return [x1_batch.astype(np.float32) / 1., x2_batch.astype(np.float32) / 1.], y_batch

    def on_epoch_end(self):
        if self.shuffle == True:
            self.x, self.label = shuffle(self.x, self.label)
            
class ValDataGenerator(tensorflow.keras.utils.Sequence):
    def __init__(self, x, label, x_real, y_real, batch_size=4, shuffle=True):
        self.x = x
        self.label = label
        self.x_real = x_real
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        

    def __len__(self):
        return int(np.floor(len(self.x) / self.batch_size))

    def __getitem__(self, index):
        x1_batch = self.x[index*self.batch_size:(index+1)*self.batch_size]
        label_batch = self.label[index*self.batch_size:(index+1)*self.batch_size]
        
        x2_batch = np.empty((self.batch_size, R, L, 1), dtype=np.float32)
        y_batch = np.zeros((self.batch_size, 1), dtype=np.float32)
        
        x1_batch, x2_batch = np.array(x1_batch), np.array(x2_batch)
        
        if self.shuffle:
            x1_batch = train_seq.augment_images(x1_batch)
        
        for i, idx in enumerate(label_batch):
            if random.random() > 0.5:
                x2_batch[i] = self.x_real[idx]
                y_batch[i] = 1.
            else:
                not_chosen_idx = random.choice(list(set(y_real) - set([idx])))
                x2_batch[i] = self.x_real[not_chosen_idx]
                y_batch[i] = 0.

        return [x1_batch.astype(np.float32) / 1., x2_batch.astype(np.float32) / 1.], y_batch

    def on_epoch_end(self):
        if self.shuffle == True:
            self.x, self.label = shuffle(self.x, self.label)
            
train_gen = DataGenerator(x_train, label_train, x_real, y_real, shuffle=True)
val_gen = ValDataGenerator(x_val, label_val, x_real, y_real, shuffle=True)
test_gen = ValDataGenerator(x_test, label_test, x_real, y_real, shuffle=True)

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

swish = tensorflow.keras.activations.swish

def square(x):
    return x ** 2

# Creating a Model

In [ ]:
x1 = layers.Input(shape=(R, L, 1))
x2 = layers.Input(shape=(R, L, 1))

inputs = layers.Input(shape=(R, L, 1))

padding = 'same'
activation = swish
pool_size = 2

feature = layers.Conv2D(32, kernel_size=3, padding=padding, strides=1)(inputs)
feature = layers.BatchNormalization()(feature) 
feature = activation(feature)
layers.Dropout(.4, input_shape=(2,))
feature = layers.MaxPooling2D(pool_size=pool_size)(feature)

feature = layers.Conv2D(64, kernel_size=3, padding=padding, strides=1)(feature)
feature = layers.BatchNormalization()(feature) 
feature = activation(feature)
layers.Dropout(.4, input_shape=(2,))
feature = layers.MaxPooling2D(pool_size=pool_size)(feature)

feature = layers.Conv2D(128, kernel_size=3, padding=padding, strides=1)(feature)
feature = layers.BatchNormalization()(feature) 
feature = activation(feature)
layers.Dropout(.4, input_shape=(2,))
feature = layers.MaxPooling2D(pool_size=pool_size)(feature)

feature = layers.Conv2D(256, kernel_size=3, padding=padding, strides=1)(feature)
feature = layers.BatchNormalization()(feature) 
feature = activation(feature)
layers.Dropout(.4, input_shape=(2,))
feature = layers.MaxPooling2D(pool_size=pool_size)(feature)

feature = layers.Conv2D(512, kernel_size=3, padding=padding, strides=1)(feature) 
feature = layers.BatchNormalization()(feature) 
feature = activation(feature)
layers.Dropout(.4, input_shape=(2,))
feature = layers.MaxPooling2D(pool_size=pool_size)(feature)

feature_model = Model(inputs=inputs, outputs=feature)

x1_net = feature_model(x1)
x2_net = feature_model(x2)

x1_net = layers.Flatten()(x1_net)
x2_net = layers.Flatten()(x2_net)
x1_net = keras.layers.UnitNormalization()(x1_net)
x2_net = keras.layers.UnitNormalization()(x2_net)
net = layers.Subtract()([x1_net, x2_net])

net = layers.Dense(16)(net)
net = square(net)
net = layers.Dense(1, activation='sigmoid')(net)

model = Model(inputs=[x1, x2], outputs=net)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', f1_m, precision_m, recall_m])

model.summary()

# 6. Train the model

- Train the defined model

In [ ]:
history = model.fit(train_gen, epochs=150, validation_data=val_gen)

# 7. Save the model
- Save the weights of the trained feature model and model.

In [ ]:
FEATURE_MODEL_PATH = 'Define the path to store the weigths of the feature model'
MODEL_PATH = 'Define the path to store the weigths of the model'

feature_model.save(FEATURE_MODEL_PATH)
model.save(MODEL_PATH)

In [ ]:
TESTSET_PATH = 'Define the path to store the testset'
np.save(TESTSET_PATH, x_test)